# Case Study: Rate My Answer!

**By: Dr. Reza Mousavi, University of Virginia, mousavi@virginia.edu**

In this notebook, we pre-process answers posted in a health forum and create classifiers that predict whether an answer is posted by a medical expert (physician) or not.

## 1. Import packages and load the data:

In [ ]:
%%capture --no-display
!pip install spacy # install spaCy
# !pip install tqdm # install tqdm package to display the progress
# !pip uninstall tensorflow -y
# !pip install tensorflow # install tensorflow for deep learning
!pip install keras # install keras for deep learning
!pip install googletrans==4.0.0-rc1
!pip install optuna

In [ ]:
import os
import sys

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

# scikit‑learn
import sklearn
from sklearn import tree
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# TensorFlow / Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import losses, layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer

# Keras standalone
from keras.models import Model, Sequential
from keras.layers import (
    LSTM, Activation, Dense, Dropout, Input,
    Embedding, Normalization, BatchNormalization
)
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from keras import layers as keras_layers
from keras.backend import clear_session
from keras.optimizers import *
from keras.utils import pad_sequences

# spaCy
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from spacy import displacy

# custom utilities
from custom_funcs import *


In [ ]:
train = pd.read_csv('train_RateMyAnswer.csv', sep = "|")
test = pd.read_csv('test_RateMyAnswer.csv', sep = "|")

### 2. Text Pre-processing

We can use spaCy's powerful tokenizer to parse our text:

In [ ]:
# Add new stop words: 
customize_stop_words = [
    'user','answer'
]

# Mark them as stop words
for w in customize_stop_words:
    nlp.vocab[w].is_stop = True
    
tqdm.pandas() # To display the progress
train['pr_answer'] = train.answer.progress_apply(lambda text: 
                                          " ".join(token.lemma_ for token in nlp(text) 
                                                   if not token.is_stop and token.is_alpha))

test['pr_answer'] = test.answer.progress_apply(lambda text: 
                                          " ".join(token.lemma_ for token in nlp(text) 
                                                   if not token.is_stop and token.is_alpha))

In [ ]:
train.head(2)

In [ ]:
sns.catplot(x="label", kind="count", palette="ch:.25", data=train)

## Over Sample 

In [ ]:
model_en_to_fr = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-fr')
tokenizer_en_to_fr = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-fr')
model_fr_to_en = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-fr-en')
tokenizer_fr_to_en = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-fr-en')

def back_translate(text):
    encoded_en = tokenizer_en_to_fr(text, return_tensors="pt", padding=True, truncation=True)
    translated_tokens = model_en_to_fr.generate(**encoded_en)
    french_text = tokenizer_en_to_fr.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    
    encoded_fr = tokenizer_fr_to_en(french_text, return_tensors="pt", padding=True, truncation=True)
    back_translated_tokens = model_fr_to_en.generate(**encoded_fr)
    back_translated_text = tokenizer_fr_to_en.batch_decode(back_translated_tokens, skip_special_tokens=True)[0]
    
    return back_translated_text


In [ ]:
if os.path.exists("augmented_train_set.pkl"):
    print("Augmented training set already saved. Loading from pickle...")
    train = pd.read_pickle("augmented_train_set.pkl")
else:

    minority_df = train[train['label'] == 1]
    
    augmented_samples = []
    for text in minority_df['pr_answer']:
        augmented_text = back_translate(text)
        augmented_samples.append(augmented_text)

    augmented_df = pd.DataFrame({
        'pr_answer': augmented_samples,
        'label': [1] * len(augmented_samples)
    })

    train = pd.concat([train, augmented_df], ignore_index=True)

    train.to_pickle("augmented_train_set.pkl")
    
    print("Augmented training set created and saved.")
    print("Original data count:", len(train))
    print("Balanced data count:", len(train))


In [ ]:
sns.catplot(x="label", kind="count", palette="ch:.25", data=train)

### 3. Deep Learning Algorithm (LSTM):

LSTM expects the data to be in a specific format. Therefore, instead of using the DTM, we use the original data to create sequences that are processed such that LSTM would accept them as inputs:

In [ ]:
X_train = train.pr_answer
X_test = test.pr_answer

le = LabelEncoder()

Y_train = train.label
Y_train = le.fit_transform(Y_train)
Y_train = Y_train.reshape(-1,1) # This is a data transformation for the LSTM model 

Y_test = test.label
Y_test = le.fit_transform(Y_test)
Y_test = Y_test.reshape(-1,1) # This is a data transformation for the LSTM model 

In [ ]:
# Get the number of unique words in pr_answer:
from collections import Counter
results = Counter()
train['pr_answer'].str.lower().str.split().apply(results.update)
print("Number of unique words in pr_answer:", len(results))

In [ ]:
# Histogram of number of words in each answer:
train['pr_answer'].str.lower().apply(lambda x: len(x.split())).hist(bins=100)

In [ ]:
def objective(trial):
    max_words = trial.suggest_int('max_words', 5000, 20000, step=500)
    max_len   = trial.suggest_int('max_len',   50,   200,   step=5)

    tok = Tokenizer(num_words=max_words)
    tok.fit_on_texts(X_train)
    seq = tok.texts_to_sequences(X_train)
    seq_matrix = pad_sequences(seq, maxlen=max_len, padding='post')

    n_layers     = trial.suggest_int('n_layers',     1,   5)
    rnn_units    = trial.suggest_int('rnn_units',   16, 256, step=16)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5, step=0.1)
    lr           = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)

    model = Sequential()
    model.add(Input(shape=(max_len,)))
    model.add(Embedding(input_dim=max_words, output_dim=64, input_length=max_len))
    for i in range(n_layers):
        return_seq = (i < n_layers - 1)
        model.add(Bidirectional(LSTM(rnn_units, return_sequences=return_seq)))
        model.add(Normalization())
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        loss='binary_crossentropy',
        optimizer=AdamW(learning_rate=lr)
    )
    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    history = model.fit(
        seq_matrix, Y_train,
        epochs=20, batch_size=32,
        validation_split=0.2,
        callbacks=[es],
        verbose=0
    )

    return min(history.history['val_loss'])

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

best = study.best_params
print("Best hyperparameters:")
print(best)

model_path = 'best_modela.pkl'
if os.path.exists(model_path):
    with open(model_path, 'rb') as f:
        final_model = pickle.load(f)
    print("Loaded existing model.")
else:
    tok = Tokenizer(num_words=best['max_words'])
    tok.fit_on_texts(X_train)
    seq = tok.texts_to_sequences(X_train)
    seq_matrix = pad_sequences(seq, maxlen=best['max_len'], padding='post')

    final_model = Sequential()
    final_model.add(Input(shape=(best['max_len'],)))
    final_model.add(Embedding(input_dim=best['max_words'], output_dim=64, input_length=best['max_len']))
    for i in range(best['n_layers']):
        return_seq = (i < best['n_layers'] - 1)
        final_model.add(Bidirectional(LSTM(best['rnn_units'], return_sequences=return_seq)))
        final_model.add(Normalization())
        final_model.add(Dropout(best['dropout_rate']))
    final_model.add(Dense(1, activation='sigmoid'))
    final_model.compile(
        loss='binary_crossentropy',
        optimizer=AdamW(learning_rate=best['learning_rate'])
    )

    es_final = EarlyStopping(monitor='val_loss', patience=10, min_delta=0.003, restore_best_weights=True)
    final_model.fit(
        seq_matrix, Y_train,
        epochs=50, batch_size=32,
        validation_split=0.2,
        callbacks=[es_final],
        class_weight={0: 1., 1: 2.},
        verbose=1
    )

    with open(model_path, 'wb') as f:
        pickle.dump(final_model, f)
    print(f"Trained and saved final model to {model_path}")


In [ ]:
max_words = 20000 # maximum number of words to be used in the analysis
max_len = 75

In [ ]:
sequences = tok.texts_to_sequences(X_train) # apply the tokenizer to the data
sequences_matrix = pad_sequences(sequences,maxlen=max_len, padding='post') 

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                                            patience = 10, # Number of epochs with no improvement after which training will be stopped
                                            min_delta=0.003, # Minimum change in the monitored quantity to qualify as an improvement
                                            restore_best_weights=True # Whether to restore model weights from the epoch with the best value of the monitored quantity
                                           )

In [ ]:
history = final_model.fit(sequences_matrix, Y_train, # Data to be used for fitting/ training the model
                    epochs=50, # Number times that the learning algorithm will work through the training data
                    batch_size=32, # Number of samples to be used in each iteration
                    verbose=1, # Whether to print the progress 
                    shuffle=True, # Shuffle the data for each epoch
                    validation_split=0.2, # The portion of samples to be used for validation (different from our test data)
                    callbacks = callback,
                    class_weight = {0: 1.,1: 2.}
                   )

In [ ]:
print(history.history.keys())
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model Loss')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

We can now prepare the test data for the network:

In [ ]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len, padding='post') # trim or pad the sentences

And finally, apply the LSTM model to the test data and evaluate it:

In [ ]:
predictionProbabilities = final_model.predict(test_sequences_matrix).flatten()
predictions =(final_model.predict(test_sequences_matrix) > 0.5).astype("int32").flatten()

In [ ]:
fpr, tpr, thresholds = roc_curve(Y_test, predictionProbabilities, pos_label = 1)

lr_auc = roc_auc_score(Y_test, predictionProbabilities)
custom_plot_roc_curve(fpr, tpr, lr_auc)
#96.223
#98.271

In [ ]:
confusionMatrix = None
confusionMatrix = confusion_matrix(Y_test, predictions)
plt.rcParams['figure.figsize'] = (5, 5)
displayConfusionMatrix(confusionMatrix)

In [ ]:
test['predicted_score'] = predictionProbabilities
test.sort_values(by = "predicted_score", ascending = False)

### 4. Apply to Kaggle data:

In [ ]:
# Read Kaggle data:
kaggle = pd.read_csv('kaggle_RateMyAnswer.csv', sep = "|")

# Pre-process the answers:
kaggle['pr_answer'] = kaggle.answer.progress_apply(lambda text: 
                                          " ".join(token.lemma_ for token in nlp(text) 
                                                   if not token.is_stop and token.is_alpha))

# Create the sequence matrix:
kaggle_sequences = tok.texts_to_sequences(kaggle['pr_answer'])
kaggle_sequences_matrix = pad_sequences(kaggle_sequences,maxlen=max_len, padding='post')

In [ ]:
kaggle['Expected'] = final_model.predict(kaggle_sequences_matrix).flatten()

In [ ]:
kaggle[['index','Expected']].to_csv("to_Kaggle_RMA.csv", index = False)

Please read the following article before this section: https://fortune.com/well/2023/04/04/chatgpt-advice-on-breast-cancer-screenings/

Here, we are going to take three questions and their corresponding answers (posted on WebMD) from https://github.com/LasseRegin/medical-question-answer-data. We then ask ChatGPT to answer the same questions in 1 paragraph. In the very last example, we ask ChatGPT the same question, but we ask it to cite real studies to support its answer. We then use our model to measure the scores in these answers: 

In [ ]:
gp_ans = pd.DataFrame({'question':["my 5 1/2-year-old son displays adhd symptoms for 20 days then for 10 days he has none. is it adhd or another condition?",
                                   "my 5 1/2-year-old son displays adhd symptoms for 20 days then for 10 days he has none. is it adhd or another condition?",
                                   "my son has add and mild autism. he has been successfully on concerta for 6+ years. can you help with his weight loss?",
                                   "my son has add and mild autism. he has been successfully on concerta for 6+ years. can you help with his weight loss?",
                                   "my son is 13 and is depressed. he has been taking vyvanse for the last 3 years and never had a problem. can you help?",
                                   "my son is 13 and is depressed. he has been taking vyvanse for the last 3 years and never had a problem. can you help?",
                                   "my son is 13 and is depressed. he has been taking vyvanse for the last 3 years and never had a problem. can you help?"],
                       
                       'answer':["adhd and bipolar mood disorder (bmd) can coexist or be mistaken for one another. bmd usually is not diagnosed until young adulthood. however studies have shown that the earlier a person is diagnosed with bmd the more likely he is to have been diagnosed with adhd previously. in this case i would just like to reiterate that there is not enough information to discuss either possibility for your son. you mentioned that he becomes hyperactive for 3 weeks but not what his behaviors are like during those 10 days. you also do not mention irritability or mood swings just adhd symptoms. keep documenting the symptoms you are concerned about including what goes on in the home and at school when you see changes in behavior (do you work those weeks does he visit a relative or have a different aide in the classroom). you also mentioned that this began 7 months ago. i would also urge you to think about what also might have changed in your son's life about that time. consulting your pediatrician or a child psychiatrist should help you unravel the mystery of your son's behavior.",
                                 "ADHD is typically characterized by consistent symptoms over time, but they can vary in different situations. It is possible that other factors, such as changes in routine, environment, or stress levels, could contribute to the fluctuation in your son's symptoms. To determine whether your son has ADHD or another condition, it is essential to consult with a healthcare professional, such as a pediatrician or child psychologist, who can provide an accurate diagnosis and appropriate guidance based on a thorough evaluation.",
                                 "stimulants in general tend to decrease appetite in certain not all children. over time this side effect seems to lessen but as in the case of your son for some children it may persist. weight loss is the result of this appetite decrease and not consuming enough calories during the day. therefore since the medication is effective and at \"the perfect dose for him \" right now i would suggest that you work on increasing your son's calorie intake. this however is often not an easy task! some suggestions include offering your son his preferred foods whenever he seems hungry; starting the day with a large calorie-laden breakfast early in the morning before the medication takes effect; increasing the calories in snacks during the day and at bedtime; and adding a high calorie supplement like ensure or shakes with ice cream etc. during the day. consult your pediatrician or a nutrionist to determine how many calories a day your son should be consuming and try to make sure this is the minimum he consumes to prevent weight loss. if you find this remains a problem i suggest you consult a pediatric nutritionist for additional suggestions. if the problems persists despite you and your son's best efforts you may want to discuss changing to a non-stimulant medication with your son's prescribing physician.",
                                 "Weight loss in children taking Concerta can be a side effect of the medication, as it may suppress appetite. To address this issue, consider offering nutritious, calorie-dense meals and snacks throughout the day, focusing on foods rich in healthy fats, proteins, and complex carbohydrates. Encourage regular mealtimes, and try to make eating enjoyable and stress-free. It's important to closely monitor your son's weight and growth, and consult with his healthcare provider for personalized recommendations and adjustments to his medication or treatment plan, if needed.",
                                 "while any of the stimulant medications can increase depressive symptoms. it's important to pay attention to this change in your son's mood and take him to a mental health professional for an assessment. children diagnosed with attention deficit/hyperactivitydisorder (adhd) at early are more likely to suffer from depression as adolescents than those who did not have adhd according to a long-term study published in the october issue of the archives of general psychiatry. although it was an uncommon occurrence the children with adhd also were somewhat more likely to think about or attempt suicide as adolescents. the study performed by researchers at the university of chicago and the university of pittsburgh followed 123 children diagnosed with adhd at age 4 to 6 for up to 14 years until they reached ages 18 to 20. it compared them with 119 children from similar neighborhoods and schools matched for age sex and ethnicity. the researchers found that 18 percent of children diagnosed early with adhd suffered from depression as adolescents about 10 times the rate among those without adhd. children with early adhd were five times as likely to have considered suicide at least once and twice as likely to have made an attempt. please get your son evaluated.",
                                 "If your son is experiencing depression while taking Vyvanse, it's crucial to consult with his healthcare provider to discuss his symptoms and evaluate whether a change in medication or dosage is necessary. Additionally, consider exploring other forms of support, such as therapy, counseling, or support groups, which can help address his emotional well-being. Encourage open communication, provide a supportive environment, and monitor any changes in his mood or behavior. Remember, it's essential to work closely with healthcare professionals to develop the most appropriate treatment plan for your son's needs.",
                                 "A study by Coghill et al. (2014) suggests that ADHD medications, including Vyvanse, can sometimes be associated with mood-related side effects. If your son is experiencing depression while taking Vyvanse, it is important to consult with his healthcare provider to discuss his symptoms and evaluate whether a change in medication or dosage is necessary. Additionally, consider exploring other forms of support, such as therapy, counseling, or support groups, which can help address his emotional well-being (Weersing et al., 2017). It's essential to work closely with healthcare professionals to develop the most appropriate treatment plan for your son's needs."],
                      'label':["webmd","chatgpt","webmd","chatgpt","webmd","chatgpt","chatgptCITE"]})
gp_ans

In [ ]:
# Pre-process the answers:
gp_ans['pr_answer'] = gp_ans.answer.progress_apply(lambda text: 
                                          " ".join(token.lemma_ for token in nlp(text) 
                                                   if not token.is_stop and token.is_alpha))

# Create the sequence matrix:
gp_ans_sequences = tok.texts_to_sequences(gp_ans['pr_answer'])
gp_ans_sequences_matrix = pad_sequences(gp_ans_sequences,maxlen=max_len, padding='post')

gp_ans['Expected'] = model.predict(gp_ans_sequences_matrix).flatten()
gp_ans

In [ ]:
gp_ans.question.iloc[2]

In [ ]:
gp_ans.answer.iloc[2]

In [ ]:
gp_ans.answer.iloc[3]

In [ ]:
gp_ans.question.iloc[4]

In [ ]:
gp_ans.answer.iloc[4]

In [ ]:
gp_ans.answer.iloc[5]

In [ ]:
gp_ans.answer.iloc[6]

References in the ChatGPT Answer:

Coghill, D., Banaschewski, T., Lecendreux, M., Soutullo, C., Johnson, M., Zuddas, A., ... & Squires, L. (2014). European, randomized, phase 3 study of lisdexamfetamine dimesylate in children and adolescents with attention-deficit/hyperactivity disorder. European Neuropsychopharmacology, 24(10), 1661-1672.

Weersing, V. R., Jeffreys, M., Do, M. T., Schwartz, K. T., & Bolano, C. (2017). Evidence Base Update of Psychosocial Treatments for Child and Adolescent Depression. Journal of Clinical Child & Adolescent Psychology, 46(1), 11-43.